## Problem 1: Geocode shopping centers (5 points)

The overall aim of problems 1-3 is to find out **how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki**.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called `shopping_centers.shp` which contains the geocoded result.

**Preparation:** Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called `shopping_centers.txt`:

 - Itis
 - Forum
 - Iso-omena
 - Sello
 - Jumbo
 - REDI
 - Tripla 
 
 *Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. 
 Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.*

`shopping_centers.txt` should have semicolon (`;`) as a separator, and the file should include the following columns:

- ``id`` (integer) containing an unique identifier for each shopping center
- ``name`` (string) of each shopping center
- ``addr`` (string) the address 


See and example of how to format the text file [in the lesson 3 materials](https://automating-gis-processes.github.io/site/master/notebooks/L3/geocoding_in_geopandas.html). Save (and upload) the text file into your exercise repository.

- Read `shopping_centers.txt` that you just created into a pandas DataFrame called ``data``:

In [1]:
# Import modules
import requests
import geojson
import geopy
import pygeos
import geopandas as gpd
from geopandas.tools import geocode
from pyproj import CRS
import pandas as pd

# Read the data (replace "None" with your own code)
data = pd.read_csv('addresses.txt', sep=";")

/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
#NON-EDITABLE TEST CELL
# Check your input data
print(data)

                                           id  \
1000                               Ruoholahti   
1001                            Kampin Keskus   
1002             Stockmann Helsingin Keskusta   
1003                              Kauppakesku   
1005                             Verkkokauppa   
1006  Kontulantie 18, 00940 Helsinki, Finland   
1007                     Kontulan Ostoskeskus   
1008                         Kmarket Puistori   
1009                              Thermoplast   
1010                         Ravintola Henrik   
1011         Helsinki Central Railway Station   
1012                              Kuparikulma   
1013                                  Kaarela   
1014                            Palmia Duetto   
1015                                Ala-Malmi   
1016                       Marja Palmujoki Ky   
1017                        Malminkartanontie   
1018                   Oulunkylän Vanha asema   
1019                            Pasilan Asema   
1020                

- Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called `geo`:

In [3]:
# Geocode the addresses using Nominatim
geo = geocode(data['addr'], provider="Nominatim", user_agent="autogis_VH", timeout=10)

In [4]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

                       geometry  \
1000  POINT (24.91556 60.16320)   
1001  POINT (24.93169 60.16902)   
1002  POINT (24.94179 60.16989)   
1003  POINT (24.97759 60.19361)   
1005  POINT (24.92160 60.15665)   
1006  POINT (11.04803 46.31448)   
1007  POINT (25.10985 60.22126)   
1008  POINT (25.03480 60.27332)   
1009  POINT (25.02880 60.26323)   
1010  POINT (24.87197 60.22244)   
1011  POINT (24.93985 60.17038)   
1012  POINT (24.88281 60.23080)   
1013  POINT (24.87855 60.24018)   
1014  POINT (24.94801 60.22179)   
1015  POINT (25.01303 60.25134)   
1016  POINT (24.89418 60.21722)   
1017  POINT (24.86690 60.25171)   
1018  POINT (24.96566 60.22982)   
1019  POINT (24.93435 60.19857)   
1020  POINT (24.86084 60.22402)   
1021  POINT (24.99362 60.24365)   
1022  POINT (25.02063 60.24355)   
1023  POINT (25.07835 60.20982)   
1024  POINT (25.13647 60.20707)   
1025  POINT (25.07530 60.22457)   
1026  POINT (25.10979 60.23785)   
1027  POINT (24.96142 60.18779)   
1028  POINT (25.0280

In [5]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

<class 'geopandas.geodataframe.GeoDataFrame'>


Check that the coordinate reference system of the geocoded result is correctly defined, and **reproject the layer into ETRS GK-25** (EPSG:3879):

In [6]:
# Check that the coordinate reference system of the geocoded result is correctly defined
geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# reproject the layer into ETRS GK-25 (EPSG:3879)
# Reproject the data
geo = geo.to_crs(epsg=3879)

In [8]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

epsg:3879


- Make a table join between the geocoded addresses (``geo``) and the original addresses (``data``) in order to link the numerical coordinates and  the `id` and `name` of each shopping center. 
- Store the output in a variable called ``geodata`` 


In [9]:
# Join the tables
geodata = geo.join(data)

In [10]:
#NON-EDITABLE TEST CELL
# Check the join output
print(geodata.head())

                              geometry  \
1000  POINT (25495311.608 6672258.695)   
1001  POINT (25496207.840 6672906.173)   
1002  POINT (25496768.622 6673002.004)   
1003  POINT (25498756.841 6675643.904)   
1005  POINT (25495645.995 6671528.068)   

                                                address  \
1000  Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...   
1001  Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...   
1002  Kauppakeskus Citycenter, 8, Kaivokatu, Keskust...   
1003  Hermannin rantatie, Hermanninmäki, Hermanni, K...   
1005  Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...   

                                id  \
1000                    Ruoholahti   
1001                 Kampin Keskus   
1002  Stockmann Helsingin Keskusta   
1003                   Kauppakesku   
1005                  Verkkokauppa   

                                               addr  
1000       Itämerenkatu 14, 00101 Helsinki, Finland  
1001          Kampinkuja 1, 00100 Helsinki, Finland  
1

In [11]:
geodata.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - nominally onshore between 24°30'E and 25°30'E but may be used in adjacent areas if a municipality chooses to use one zone over its whole extent.
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

- Save the output as a Shapefile called `shopping_centers.shp` 

In [12]:
# Define output filepath
out_fp = "shopping_centers.shp"

# Save file
geodata.to_file(filename=out_fp)

In [13]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Geocoded output is stored in this file: shopping_centers.shp


## Problem 2: Create buffers around shopping centers (5 points)

Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points. 


- Start by creating a new column called `buffer` to ``geodata`` GeoDataFrame:

In [14]:
geodata['buffer'] = ''

- Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new `buffer` column.

Here, you can use the [GeoDataFrame buffer() method](http://geopandas.org/geometric_manipulations.html#GeoSeries.buffer), which uses Shapely's [buffer](http://toblerity.org/shapely/manual.html#object.buffer) in the bacground. You only need to use the `distance` -parameter, don't worry about the other parameters.

In [15]:
geodata['buffer'] = geodata['geometry'].apply(lambda x: x.buffer(1.5))

In [16]:
geodata = geodata.reset_index(drop=True)

In [17]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                           geometry  \
0  POINT (25495311.608 6672258.695)   
1  POINT (25496207.840 6672906.173)   
2  POINT (25496768.622 6673002.004)   
3  POINT (25498756.841 6675643.904)   
4  POINT (25495645.995 6671528.068)   

                                             address  \
0  Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...   
1  Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...   
2  Kauppakeskus Citycenter, 8, Kaivokatu, Keskust...   
3  Hermannin rantatie, Hermanninmäki, Hermanni, K...   
4  Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...   

                             id  \
0                    Ruoholahti   
1                 Kampin Keskus   
2  Stockmann Helsingin Keskusta   
3                   Kauppakesku   
4                  Verkkokauppa   

                                            addr  \
0       Itämerenkatu 14, 00101 Helsinki, Finland   
1          Kampinkuja 1, 00100 Helsinki, Finland   
2           Kaivokatu 8, 00101 Helsinki, Finland   
3  Herman

In [ ]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

In [ ]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

- Replace the values in `geometry` column with the values of `buffer` column:

In [18]:
# Replace the values in geometry column with the values of buffer column:
geodata['geometry'] = geodata['buffer']

In [19]:
#NON-EDITABLE TEST CELL
print(geodata.head())

                                            geometry  \
0  POLYGON ((25495313.108 6672258.695, 25495313.1...   
1  POLYGON ((25496209.340 6672906.173, 25496209.3...   
2  POLYGON ((25496770.122 6673002.004, 25496770.1...   
3  POLYGON ((25498758.341 6675643.904, 25498758.3...   
4  POLYGON ((25495647.495 6671528.068, 25495647.4...   

                                             address  \
0  Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...   
1  Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...   
2  Kauppakeskus Citycenter, 8, Kaivokatu, Keskust...   
3  Hermannin rantatie, Hermanninmäki, Hermanni, K...   
4  Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...   

                             id  \
0                    Ruoholahti   
1                 Kampin Keskus   
2  Stockmann Helsingin Keskusta   
3                   Kauppakesku   
4                  Verkkokauppa   

                                            addr  \
0       Itämerenkatu 14, 00101 Helsinki, Finland   
1          

Optional: at this point, you can drop out unnecessary columns from the geodataframe. In the next problem, 
we will only need these columns: `'id', 'name', 'geometry'`

In [20]:
geodata = geodata[['id', 'address', 'geometry']]

In [21]:
geodata.rename(columns={'id':'name'}, inplace=True)

In [22]:
geodata.head()

,name,address,geometry
0,Ruoholahti,"Ruoholahti, 14, Itämerenkatu, Ruoholahti, Läns...","POLYGON ((25495313.108 6672258.695, 25495313.1..."
1,Kampin Keskus,"Kamppi, 1, Kampinkuja, Kamppi, Eteläinen suurp...","POLYGON ((25496209.340 6672906.173, 25496209.3..."
2,Stockmann Helsingin Keskusta,"Kauppakeskus Citycenter, 8, Kaivokatu, Keskust...","POLYGON ((25496770.122 6673002.004, 25496770.1..."
3,Kauppakesku,"Hermannin rantatie, Hermanninmäki, Hermanni, K...","POLYGON ((25498758.341 6675643.904, 25498758.3..."
4,Verkkokauppa,"Hesburger, 9, Tyynenmerenkatu, Jätkäsaari, Län...","POLYGON ((25495647.495 6671528.068, 25495647.4..."


## Problem 3: How many people live near shopping centers? (5 points)

Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out **how many people live near each shopping center**. We will use the same data as we did during [lesson 3](https://automating-gis-processes.github.io/site/notebooks/L3/spatial-join.html#Spatial-join); **a Population Grid** that is available via the HSY wfs. 

Alternatively, you can also download the data from the [Helsinki Region Infoshare (HRI)](https://www.hsy.fi/fi/asiantuntijalle/avoindata/Sivut/AvoinData.aspx?dataID=7) as a shapefile (using wget).

The coordinate reference system of the population grid is **ETRS GK-25 (EPSG:3879)**.


**Steps:**

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: ``'asukkaita'`` (=population count per grid square) and ``'geometry'`` 

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer **into the population grid layer**

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

**Finally:**

- Print out the population living within 1.5 km from each shopping center:

     - Itis
     - Forum
     - Iso-omena
     - Sello
     - Jumbo
     - REDI
     - Tripla
     
**Final print out should contain both the shopping center name and population count**, for example: `25858 people live within 1.5 km from Iso-Omena`.

*Feel free to divide your solution into several codeblocks! Remember to comment your code  :)*

In [23]:
# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# # Read population grid data for 2018 from geojson into a variable `pop` as geoDataFrame
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [24]:
# Check the crs info
pop.crs = CRS.from_epsg(3879).to_wkt()

In [25]:
geodata.crs == pop.crs

True

In [26]:
#NON-EDITABLE TEST CELL
# Check your input data
print("Number of rows:", len(pop))
print(pop.head(3))

Number of rows: 3167
                                            geometry  index  asukkaita  \
0  MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...   3342        108   
1  MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...   3503        273   
2  MULTIPOLYGON Z (((25476999.994 6675749.004 0.0...   3660        239   

   asvaljyys  ika0_9  ika10_19  ika20_29  ika30_39  ika40_49  ika50_59  \
0         45      11        23         6         7        26        17   
1         35      35        24        52        62        40        26   
2         34      46        24        24        45        33        30   

   ika60_69  ika70_79  ika_yli80  
0         8         6          4  
1        25         9          0  
2        25        10          2  


In [27]:
gpd.options.use_pygeos = True

/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [28]:
# Change the name of a column
pop = pop.rename(columns={'asukkaita': 'pop18'})

In [29]:
# Subset columns
pop = pop[["pop18", "geometry"]]

In [30]:
# Create a spatial join between grid layer and buffer layer, only geodata addresses falling within pop will get returned!
join = gpd.sjoin(geodata, pop, how="inner", op="within")

#### Did some addresses fall outside of our grid?

In [32]:
len(geodata) > len(join)

True

#### Report how many people live within 1.5 km distance from each shopping center

In [34]:
join.pop18.sum()

14461

In [35]:
geodata.to_file('shopping_centers_ouput.txt')

**Reflections:**
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

YOUR ANSWER HERE

Well done! Now you can continue to [problem 4](Exercise-3-Problem-4.ipynb)